## AdaBoost

原理参考

[知乎：手把手教你AdaBoost](https://zhuanlan.zhihu.com/p/27126737?utm_source=wechat_session&utm_medium=social&utm_oi=1044170377731248128)

In [5]:
import numpy as np
import pandas as pd
x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
len(x)

10

In [6]:
y = [1, 1, 1, -1, -1, -1, 1, 1, 1, -1]
y

[1, 1, 1, -1, -1, -1, 1, 1, 1, -1]

基学习算法：二分类器，就给个阈值

### 第一个个体学习器

我们首先认为$x_i$(i=1,2,…,10)的权重是一样的，即每一个数据同等重要。（权重是用来计算误差的）

In [7]:
w_1 = []
for i in range(len(x)):
    w_1.append(0.1)
w_1

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]

In [8]:
data = {'x':x, 'y':y,'w_1':w_1}
pd.DataFrame(data).T

,0,1,2,3,4,5,6,7,8,9
x,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
y,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
w_1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1


In [9]:
# 设置阈值
threshs = [i - 0.5 for i in x]
threshs.append(9.5)
threshs

[-0.5, 0.5, 1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5, 8.5, 9.5]

In [10]:
# 小于阈值的为1，大于阈值的为-1
# 根据每个阈值进行预测
# 以thresh = 2.5 为例
y_pres = []
thresh = 2.5
for i in range(len(x)):
    if x[i] < thresh:
        y_pres.append(1)
    else:
        y_pres.append(-1)
y_pres

[1, 1, 1, -1, -1, -1, -1, -1, -1, -1]

In [11]:
y_pres_all_thresh = {}
for thresh in threshs:
    y_pres = []
    for i in range(len(x)):
        if x[i] < thresh:
            y_pres.append(1)
        else:
            y_pres.append(-1)
    y_pres_all_thresh[thresh] = y_pres
pd.DataFrame(y_pres_all_thresh).T


,0,1,2,3,4,5,6,7,8,9
-0.5,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
0.5,1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1.5,1,1,-1,-1,-1,-1,-1,-1,-1,-1
2.5,1,1,1,-1,-1,-1,-1,-1,-1,-1
3.5,1,1,1,1,-1,-1,-1,-1,-1,-1
4.5,1,1,1,1,1,-1,-1,-1,-1,-1
5.5,1,1,1,1,1,1,-1,-1,-1,-1
6.5,1,1,1,1,1,1,1,-1,-1,-1
7.5,1,1,1,1,1,1,1,1,-1,-1
8.5,1,1,1,1,1,1,1,1,1,-1


In [13]:
# 计算每个阈值的错误率
e = 0
thresh = 1.5
for i in range(len(y)):
    if y_pres_all_thresh[thresh][i] != y[i]:
        e+= w_1[i]
e

0.4

In [15]:
# 计算所有阈值的错误率
e_s = {}
e_min = len(x)
n = 0
for thresh in threshs:
    e = 0
    for i in range(len(y)):
        if y_pres_all_thresh[thresh][i] != y[i]:
            e+= w_1[i]
    e_s[thresh] = round(e,6)
    if e < e_min:
        e_min = round(e, 6)
        n = thresh
e_s

{-0.5: 0.6,
 0.5: 0.5,
 1.5: 0.4,
 2.5: 0.3,
 3.5: 0.4,
 4.5: 0.5,
 5.5: 0.6,
 6.5: 0.5,
 7.5: 0.4,
 8.5: 0.3,
 9.5: 0.4}

In [16]:
# 最小错误率
e_min

0.3

In [17]:
# 最小错误率对应阈值（同样错误率选小的阈值）
n

2.5

In [18]:
data['pre_1'] = y_pres_all_thresh[n]
pd.DataFrame(data).T

,0,1,2,3,4,5,6,7,8,9
x,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0
y,1.0,1.0,1.0,-1.0,-1.0,-1.0,1.0,1.0,1.0,-1.0
w_1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1
pre_1,1.0,1.0,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [20]:
# 计算模型权重a
import math

a_1 = 0.5 * math.log((1 - e_min)/e_min)
a_1 = round(a_1, 6)
a_1

0.423649

这里可以看出，错误率为什么必须小于0.5，否则a<0

In [23]:
# 更新权重

w_2_tmp = []

for i in range(len(x)):
    w_new = w_1[i]*math.exp(-1*a_1*y[i]*y_pres_all_thresh[n][i])
    w_new = round(w_new, 6)
    w_2_tmp.append(w_new)

w_2_tmp

[0.065465,
 0.065465,
 0.065465,
 0.065465,
 0.065465,
 0.065465,
 0.152753,
 0.152753,
 0.152753,
 0.065465]

In [24]:
z = 0
for i in range(len(w_2_tmp)):
    z += w_2_tmp[i]
z

0.916514

In [25]:
z = sum(w_2_tmp)
z

0.916514

In [26]:
w_2 = [round(i/z, 5) for i in w_2_tmp]
w_2

[0.07143,
 0.07143,
 0.07143,
 0.07143,
 0.07143,
 0.07143,
 0.16667,
 0.16667,
 0.16667,
 0.07143]

可以看到x=6,7,8的数据的权重变大了，而其他数据的权重降低了，这是希望能把之前经常分类错误（经常分类错误会出现权重不断变大）的数据能在下一个个体学习器分类正确（记住：权重是用来计算误差的，为了降低误差，选择阈值时会倾向把权重大的分类正确）


In [27]:
data['pre_1']

[1, 1, 1, -1, -1, -1, -1, -1, -1, -1]

In [28]:
g_1_s = []
for i in range(len(x)):
    g_1 = a * data['pre_1'][i]
    g_1_s.append(g_1)
data['g_1'] =  g_1_s
pd.DataFrame(data).T

NameError: name 'a' is not defined

In [29]:
def func_AdaBoost(x, y, T):
    
    # 储存变量的列表
    a_n_s = [] # 存放个体学习器权重
    w_n_s = [] # 存放样本权重
    pre_n_s = [] # 存放预测值
    g_n_s = []  # 存放加权后预测值
    n_s = []

    
    # 权重初始化
    w_1 = func_w_1(x)    
    w_n_s.append(w_1)
    
    # 生成阈值
    threshs = func_threshs(x)
    
    for i in range(3):  
    
        # 变量权重
        w_n = w_n_s[i]
        # 所有阈值二分法结果 positive 小于阈值为1，大于阈值为-1，negative相反
        y_pres_all_thresh_tmp_ps, y_pres_all_thresh_tmp_ng = func_y_pres_all_thresh_tmp(threshs)
        # 计算一个阈值尝试两种判定，计算所有情况加权后错误率，最小错误率，及对应的阈值
        e_s, e_min, n, y_pres_all_thresh = func_e_s(x, y, threshs, w_n, y_pres_all_thresh_tmp_ps, y_pres_all_thresh_tmp_ng)
        # 获得最佳阈值对应预测
        pre_n = func_pre_n(y_pres_all_thresh, n)
        # 计算模型权重a
        a_n = func_a_n(e_min)
        # 计算权重暂时值
        w_n_tmp = func_w_n_tmp(x, pre_n,w_n,a_n)
        # 计算z
        z_n = func_z_n(w_n_tmp)
        # 计算调整后权重
        w_n = func_w_n(w_n_tmp, z_n)
        # 计算第一个模型加权后的预测
        g_n = func_g_n(x, pre_n, a_n)
        
        # 数据存储
        a_n_s.append(a_n) # 存放个体学习器权重
        w_n_s.append(w_n)# 存放样本权重
        pre_n_s.append(pre_n)# 存放预测值
        g_n_s.append(g_n)
        n_s.append(n)

        
    return a_n_s, w_n_s, pre_n_s, g_n_s, n_s